**This can generate Gaussian Random Field For a given APS.**

**Functions:**

- `APS_func`

- `GRF_hpmap_gen`

In [1]:
#Import Necessary Libraries
import numpy as np
import scipy as sp 
import healpy as hp
import builtins as blt
from functools import partial,lru_cache
import numexpr as ne
import time
import os
from astropy.table import Table
from astropy.io import fits

#### APS_func

**The aim of this is to produce the Gaussian Random Field from a given input Angular Power Spectrum**

This is the input model Angular Power Spectrum(APS) Function.
    
$$C_{\ell}=Amp \ \ell^{\beta}$$
    
Amp here defines the Amplitude of APS.beta($\beta$) here is the power index.

**Parameters**

l : `float or array` 
    The Angular Multipole value.
        
**Returns**
    `float or array` 
    The Angular Power Spectrum Value for that multipole(s).


In [2]:
def APS_func(l):  
   return Amp * np.power(l, beta)



#### GRF_hpmap_gen

Generates The GRF(Gaussian Random Field for all the pixels) for the input APS.

**Parameters**

**nside** : `int`  
        Healpix parameter Usually power of 2.
        
**l_max** : `int` 
        Maximum Value of Angular Multipole.
        
**nu** :  `float` 
        The Observing frequency $\nu$ in MHz.
        
**func** : `function` 
        The input APS.
        
**iseed** : `int or float` 
        A seed value.In order to get different realizations for same APS.
        
**Returns**

`array` 
    Contains the GRF value for each pixels.Shape $1 \times12\ {nside}^2$.
   


In [3]:
def GRF_hpmap_gen(nside, l_max, nu, func, iseed = -1):
    kk= np.zeros(l_max)
    kk[1:]=func(np.arange(1,l_max, dtype=np.float64))
    if (iseed != -1):
        np.random.seed(iseed)
        #print(f"iseed={iseed}")
    map_grf = hp.synfast(kk,nside=nside, lmax=l_max)
    return map_grf


**This contains useful parameters of MWA.**

`The Important things are listed here:`

- `KB` - Boltzmann Constant in $Jy\ m^2/mK$

- `nside` - Healpix parameter.

- `Amp` - Amplitude of APS

- `beta` - Power index.

- `Nrea` - No of Realizations.

- `ra_ptg` - Pointing direction of telescope.

- `b` - Dimensions of Square Aperture in m.

- `nu_c` - Central Frequency.

- `dec_mwa` - Declination of the telescope.


In [5]:
# ======== constants ========#
blt.C = 299.792  # velocity of light in Km/s

def paramsim():
    """
    This Function contains the simulation parameters.
    """
    blt.KB = 1.38		 	# Boltzman constant in Jy.m^2/mK
    #blt.pi=np.pi
    # Healpy parameters for simulation
    blt.nside =32  # Nside for Healpix maps which sets the resolution
    
    # Input < C_{\ell}= Amp * \ell^{\beta} >parameters for simulation
    blt.Amp = 100  # amplitude of the input angular power spectrum in mK^2
    blt.beta = -2
    
     # No of realizations
    blt.Nrea = 10 # number of realizations of the simulations
    blt.ra_ptg=0
    
    # # observation input parameters
    blt.ra_ini = 0  # initial pointing RA of MWA during observation in degree
    # initial phase center of observation w.r.t. dec of MWA (-26.7 deg)
    blt.dec_ini = 0.0
    blt.obs_time = 0.0  # total observation time in mins
    blt.ptg_time = 0.0  # pointing time in secs



In [6]:
def params(tele):
	if (tele == 'MWA'):
		#MWA system params:
		blt.b = 4.0				#antenna dimensions in meters
		blt.nu_c = 154.25 		#central frequency in MHz
		#blt.Nchan = 154.24			# No. of channel
		blt.B_bw = 30.72			#in MHz units bandwidth
		#blt.dnu_c = blt.B_bw/blt.Nchan 		# channel width in MHz units
		#blt.lam_c = C/nu_c 		# Wavelength corresponding to central frequency in meter
		blt.NA = 126			#no of antennas
		blt.Nbl = NA*(NA-1)/2	#no of baselines
		blt.A_dBdT = 1			#(A*(dB/dT)^2)-1 (mk/Jy)^2 # 4 factor due to modified pbeam
		blt.dec_mwa = -26.79522222222223		#over head declination of MWA in degree
#		blt.Tsys = 150 			# system temperature in K
#		blt.eta = 0.6		 	# efficiency of the telescope	
	
	

			
		#Cosmology of MWA	
#		blt.r = 6845.5			#comoving distance in Mpc
#		blt.rp = 11.5			#dr/dnu in Mpc/MHz
#		blt.dBdT = 3.27			#(dB/dT)_326.5 in Jy/mK 
	
		
	else :
		print ("Please enter telescope name")
	return (0)

**This can Generate Primary Beam Pattern of MWA**.

**Functions:**

- `needful_pixels`

- `hat_n`

- `beam_mwa`

- `Primary_Beam_generate`

- `dot_cal_superfast`

- `calculate_phase`


#### needful_pixels
This calculates the needful pixels which makes less than radius radian w.r.t the pixel in the direction given by ra_ptg,dec_ptg.

**Parameters**
   
**ra_ptg** : `int or float` 
        RA of the pointing direction.(In Degrees).
        
**dec_ptg** :  `int or float` 
        DEC of the pointing direction.(In Degrees).
        
**nside** : `int` 
        Healpix parameter.Usually Power of 2.
        
**radius**  : `int or float` 
        The maximum angle subtended by a pixel given the pixel center by ra_ptg,dec_ptg.(In Radians)
        By Default 90 degrees only upper hemisphere.
        
**Returns**
    
**pix_mask** : `array` 
        The pixels lie within the given disk radius.Shape $6\ {nside}^2 \times 1$.

In [7]:
@lru_cache(maxsize=None)
def needful_pixels(ra_ptg,dec_ptg,nside,radius=np.radians(90)):
    MWA_vec = hp.ang2vec(ra_ptg, dec_ptg, lonlat=True)
    ipix_mask = hp.query_disc(nside=nside, vec=MWA_vec, radius=np.radians(90))
    return ipix_mask

#### hat_n
This can calculate the $xyz$ components of $\hat{n}$ for all the pixels which makes angle less
than 90 deg by default w.r.t. the pixel in the direction given by ra_ptg,dec_ptg.

**Parameters**
    
**nside** : `int` 
    Healpix parameter.Usually Power of 2.
        
**ra_ptg** : `int or float` 
    RA of the pointing direction.(In Degrees).
        
**dec_ptg** : `int or float` 
    DEC of the pointing direction.(In Degrees).

**Returns**
    `Array` 
    Containg the $xyz$ comps of $\hat{n}$. $6\ {nside}^2 \times 1$.


In [8]:
@lru_cache(maxsize=None)
def hat_n(nside,ra_ptg,dec_ptg):
    theta, phi = hp.pix2ang(nside,needful_pixels(ra_ptg,dec_ptg,nside))
    hat_n=np.array([np.sin(theta) * np.cos(phi),np.sin(theta) * np.sin(phi),np.cos(theta)]).T
    return hat_n

#### beam_mwa
This is the primary beam function of the MWA Telescope.
    
$$A(\hat{n},\nu) = sinc^2\left(\frac{{\pi}b{\nu}\ \hat{n}\cdot\hat{e}_1 (\alpha_{p})}{c}\right)sinc^2\left(\frac{{\pi}b{\nu}\ \hat{n}\cdot\hat{e}_2 (\alpha_{p})}{c}\right)$$

    
**Parameters**
  
**nu** : `float` 
    Observing Frequency $\nu$ in MHz unit.
    
**ne1** : `float or array` 
    The dot product by $\hat{n}\cdot\hat{e}_ 1$.
    
**ne2** : `float or array` 
    The dot product by $\hat{n}\cdot\hat{e}_ 2$.
    
**Returns**
 
`float or array` 
    The primary beam pattern value for square aperture(MWA).


In [9]:
def beam_mwa(nu,ne1,ne2):
    return (np.sinc((b*nu/C)*ne1)**2.0)*(np.sinc((b*nu/C)*ne2)**2.0)

#### Primary_Beam_generate
This is the main function which can generate the Primary Beam (for all the pixels which makes angle less
than 90 deg by default w.r.t. the pixel in the direction given by a0,d0) for MWA given the RA,DEC and frequency.

**Parameters**
  
**nside** : `int` 
    Healpix parameter.Usually Power of 2.
    
**a0** : `int or float` 
    RA of the pointing direction.(In Degrees)
    
**d0** : `int or float` 
    DEC of the pointing direction.(In Degrees)
    
**nu** : `float` 
    Observing Frequency $\nu$ in MHz unit.
    
**beam_mwa** : `function` 
    The PB pattern of MWA.
    
**Returns**
    
**PB** : `array` 
    The value of Primary beam of The Telescope.Shape $6\ {nside}^2 \times 1$.


In [10]:
def Primary_Beam_generate(nside,a0,d0,nu,beam_mwa):
    start=time.time()
    hatn=hat_n(nside,a0,d0)
    #convert RA,Dec into radians
    a0=np.deg2rad(a0)   #RA of the phase center in radians
    d0 = np.deg2rad(d0) #Phase center dec of the observation  in radians
    #create the basis vectors
    e1 = np.array([-np.sin(a0), np.cos(a0), 0])
    e2 = np.array([-np.cos(a0)*np.sin(d0), -np.sin(d0)*np.sin(a0), np.cos(d0) ])
    e3 = np.array([np.cos(a0)*np.cos(d0), np.sin(a0)*np.cos(d0), np.sin(d0) ])
    # Calculate the dot product e1 . hat_n for all pixels
    ne1 = np.dot(hatn,e1) #n⋅e1
    # Calculate the dot product e2 . hat_n for all pixels
    ne2 = np.dot(hatn,e2) #n⋅e2
    PB=beam_mwa(nu,ne1,ne2)
    end=time.time()
    
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f"Time taken To calculate Beam Pattern for RA {np.degrees(a0)} , Dec {np.degrees(d0)}")
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))
    
    return PB 

#### dot_cal_superfast
This function can calculate the $xyz$ component of $\hat{n}$ for all the pixels which makes angle less
than 90 deg by default w.r.t. the pixel in the direction given by ra_ptg,dec_ptg 
along the basis vectors $\hat{e_1},\hat{e_2},\hat{e_3}$ given by the RA,DEC.

**Parameters**
 
**nside** : `int` 
        Healpix parameter.Usually Power of 2.
        
**ra_ptg** : `int or float` 
        RA of the pointing direction.(In Degrees)
        
**dec_ptg** :`int or float` 
        DEC of the pointing direction.(In Degrees)
        
**Returns**
    
**dot_products** : `array` 
        Contains the $xyz$ comps for those pixels.Shape $6\ {nside}^2 \times 3$.


In [11]:
def dot_cal_superfast(nside,ra_ptg,dec_ptg):
    #convert the RA,Dec into radians
    a0=np.deg2rad(ra_ptg)   #RA of the phase center in radians
    d0 = np.deg2rad(dec_ptg) #Phase center dec of the observation  in radians
    #create the basis functions e1,e2,e3
    e1 = np.array([-np.sin(a0), np.cos(a0), 0])
    e2 = np.array([-np.cos(a0)*np.sin(d0), -np.sin(d0)*np.sin(a0), np.cos(d0)])
    e3 = np.array([np.cos(a0)*np.cos(d0), np.sin(a0)*np.cos(d0), np.sin(d0)])
    e=np.stack((e1,e2,e3),axis=1)
    dot_products = np.dot(hat_n(nside,ra_ptg,dec_ptg),e)
    return dot_products

#### calculate_phase
Given the baselines it can calculate the phase factor for
for all the pixels which makes angle less than 90 deg by default w.r.t. the pixel in 
the direction given by ra_ptg,dec_ptg of the Telescope.

**Parameters**
  
**dot_product** : `array`  
        Contains the $xyz$ comps for those pixels that makes less than 90 degree angle. 
        
**bl** : `array` 
        The array conatins the components of the baselines.(defined by basis vectors $\hat{e_1},\hat{e_2},\hat{e_3}$).
        Shape $N_{Baselines}\times 3$
        
**Returns**
 
`array` 
        The phase factor . Shape $6\ {nside}^2 \times  N_{Baselines}$.
        Excluding the factor $e^{-2 \pi i \vec{U} \cdot \hat{p}}$ 
        Which will be multiplied later on.


In [12]:
def calculate_phase(dot_product,bl):
    dot=dot_product@bl.T
    ppi=np.pi
    return ne.evaluate('exp(2*ppi*1j*dot)') #np.exp(2j*np.pi*dot).astype(np.complex64)

#### visgen_mwa_multi
This is the main module which calculates the visibilities for all the baselines
given the RA,DEC,Frequency,GRF map and the baseline files.(Also does Multiple Realizations)

**Parameters**
 
**nside** : `int` 
        Healpix parameter.Usually Power of 2.
        
**in_sky_map** : `array` 
        Contains the Sky Brightness Temperature simulated from model APS for all the pixels.(Can contain Multiple Realizations).Shape $N_{Realizations}\times 12\ {nside}^2$.
        
**ra_ptg** : `int or float` 
        RA of the pointing direction.(In Degrees)
        
**dec_ptg** : `int or float` 
        DEC of the pointing direction.(In Degrees)
        
**bl_file** : `array` 
        The array conatins the components of the baselines.(defined by basis vectors $\hat{e_1},\hat{e_2},\hat{e_3}$).Shape $N_{Baselines}\times 3$.
        
**nu** : `float` 
        Observing Frequency $\nu$ in MHz unit.
        
**chunk_size** : `int`
        No of baselines for which visibilities are being computed in a single step.
        chunk_size=80 (Default set),adjust the chunk size as per system requirements.
        For higher core systems it can be made high say 500.

**Returns**
    
**vis** : `Complex array` 
        Contains the simulated visibility for all the baselines given by bl_file.(Along axis=0 is the different Realizations,axis=1 is the baselines).Shape $N_{realizations}\ \times\ N_{Baselines}$.
    


In [13]:
def visgen_mwa_multi(nside, in_sky_map, ra_ptg, dec_ptg, bl_file, nu,chunk_size=80):
    start = time.time()
    Npix = hp.nside2npix(nside)     #Number of pixels given N_side
    res = hp.nside2resol(nside, arcmin = True)      #resolution given N_side
    lmax = nside * 3 - 1        #maximum number of \ell multipoles used
    print("Nside=",nside, 'resolution=', res, 'min', Npix, 'pixels' )
    dOmega = hp.nside2resol(nside)**2.0  # resolution in s.rad
    Pb_map=Primary_Beam_generate(nside,ra_ptg,dec_ptg,nu_c,beam_mwa)
    dot_product=dot_cal_superfast(nside,ra_ptg,dec_ptg)
    lam = C/nu  # wavelength in m
    print(lam)
    Q_nu = 2.0 * KB / lam**2.0  # in Jy/mK
    print(Q_nu)
    ipix_mask= needful_pixels(ra_ptg,dec_ptg,nside)
    GRF_PB_Product =in_sky_map[:,ipix_mask]*Pb_map
    print(GRF_PB_Product.shape)
    bl_file=bl_file/lam #convert to baseline unit
    nbl, nn = bl_file.shape  # number of baselines
    chunks = np.array_split(np.arange(nbl), np.arange(chunk_size, nbl, chunk_size))
    #print(chunks)
    vis = np.zeros((Nrea, nbl), dtype=np.complex128)
    for ii in chunks:
        phase=calculate_phase(dot_product,bl_file[ii,:]) #phase calculation
        vis[:, ii]=GRF_PB_Product@phase
        #print(ii)
    vis =  Q_nu * dOmega * vis 
    #compensate for -u dot p term for all baselines
    ppi=np.pi
    w=bl_file[:,2]
    vis=ne.evaluate('vis*(exp(-2.0*ppi*1j*w))')
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))  
    return vis

This is the file which calls the other modules
This can simulate the visibilities for a given Baseline file.
The Visibility is given by

$$\mathcal{V}(\vec{U},\nu)=Q_\nu \int_{UH}\ d\Omega_{\hat{n}}\ T(\hat{n},\nu) \ A(\Delta\hat{n},\nu)\ e^{2\pi i\vec{U}\cdot\Delta\hat{n}}$$

In Healpix we discretize the sky and the integral becomes summation.But the integral is restricted to only upper hemisphere.
So we find out those pixel indexing and sum only over them.

**The discritized version is given as**:

$$\mathcal{V}(\alpha_{p},\vec{U},\nu) = Q_{\nu}\ \Delta\Omega_{pix}\ \sum_{q} \ T(\hat{n}_{q},\nu) \ A(\Delta\hat{n}_{q},\nu) \ e^{2\pi i\vec{U}\cdot\Delta\hat{n}}$$
    
    
where

\begin{align}
Q_{\nu} &=2k_B /\lambda^2\\
\Delta\hat{n}&= \hat{n}- \hat{p}\\
\vec{\text{U}} &= u\ {\hat{e}_ 1 (\alpha_{p})} + v \ {\hat{e}_ 2 (\alpha_{p})} + w \ {\hat{e}_3(\alpha_{p})}
\end{align}


Where $\Delta\Omega_{pix}$ refers to the solid angle subtended by each simulation pixel and $\hat{p}$ refers to the pointing direction of the antenna.Here $\hat{p}=\hat{e}_3$.

**Overview**

- First we generate the **Gaussian Random Field** from given **Angular Power Spectrum**.

- Then We find out Which Pixels are in the Upper Hemisphere given by the pointing direction.Here Pointing Direction is vertically overhead.

- Calculate the **Primary Beam Pattern** for the Telescope.

- Calculate the components of $\hat{n}$ along the basis vectors $\hat{e}_1,\hat{e}_2,\hat{e}_3$.This will help us to determine the dot product $\vec{\text{U}}\cdot\hat{n}$.

- Now We calculate the **Phase factor** $e^{2\pi i\vec{\text{U}}\cdot\hat{n}}$.

- Multiply the **GRF and PB** first and then multiply by phase factor ,sum over the pixels.The whole thing is `done by matrix multiplication.`

- Finally multiply by $e^{-2\pi i\vec{\text{U}}\cdot\hat{n}}=e^{-2\pi i w}$,so that we get the correct Phase Factor.

- The Visibility is Simulated.

The **Basis** vectors $\hat{e_1},\hat{e_2},\hat{e_3}$ are dependent on the R.A. $\alpha$ and DEC $\delta$ \
and the components of them in the cartesian system $xyz$ is given by:

\begin{align}
\hat{e_1}&=[-\sin{\alpha},\cos{\alpha},0]\\
\hat{e_2}&=[-\cos{\alpha}\sin{\delta},-\sin{\alpha}\sin{\delta},\cos{\delta}]\\
\hat{e_3}&=[\cos{\alpha}\cos{\delta},\sin{\alpha}\cos{\delta},\sin{\delta}]
\end{align}




In [18]:
#call the paramertes
paramsim()
params("MWA")

#check UAPS or APS
if (Amp == 1) and (beta == 0):
    uaps = True
else:
    uaps = False
if uaps:
    print('UAPS simulation')
else:
    print(f'APS simulation for Amp = {Amp} and beta = {beta}')

Npix = hp.nside2npix(nside)
start=time.time()

#Generate the GRF
map_grf = np.zeros((Nrea, Npix))
for ii in range(Nrea):
    iseed = 10*ii+1 # 0, 11, ... etc. iseed must chane with realizations
    map_grf[ii] =GRF_hpmap_gen(nside = nside, l_max =nside * 3 - 1 , 
                                   nu = nu_c, func =APS_func, iseed = iseed) # -1 for random
print(f"{Nrea} Sky maps generated.")

end1 = time.time()
hours, rem = divmod(end1-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Took time for generating GRF",end= '\t ')
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))
print(map_grf.shape)


# # working with the input file
# hdulist = fits.open("1000000000.fits",mode='readonly')
# # print (hdulist.info()) 				#to check different headers
# data_tmp = hdulist[0].data 				#to save visibilities in data
# dataT = Table(data_tmp)                 #to convert data from array form to table form, readable

# nu_c = hdulist[0].header['CRVAL4']*1.e-6   #Centered Frequency in MHz unit 1e-6 to convert from Hz to MHz
# pol = hdulist[0].header['NAXIS3']          #No of Polarization Channels Stokes I,U,V,W
# Nchan = hdulist[0].header['NAXIS4']        #No of Frequnecy Channels
# dnu_c = hdulist[0].header['CDELT4']*1.e-6  #Frequency Resolution in MHz unit 1e-6 to convert from Hz to MHz

# #EXTRACT the baselines
# u = dataT['UU']*3.e8 #*nu_c for baseline unit   #*3.e8 (for meter unit)
# v = dataT['VV']*3.e8
# w = dataT['WW']*3.e8
# bln = np.stack((u,v,w),axis = 1)
# print(bln.shape)
# hdulist.close()
print('RA = %s'%ra_ptg)
print('DEC = %s'%dec_mwa)

#for loading from a file
bln=np.load("MWA_BASELINE_7875.npy")
print(bln.shape)

#generate the visibilities 
vis=visgen_mwa_multi(nside, map_grf, ra_ptg, dec_mwa, bln,nu_c)
print(vis.shape)



#save file in a directory
output_dir = "./simulated_visibility"
#check if directory exists,if not make one
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#If you want to save it in a .npy file uncomment this
if uaps:
    op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_UAPS_jupyter"
else:
    op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_APS_{Amp}_beta{beta}_jupyter"

np.save(op_filename,vis)

#data output for multi channel multiple realization different file
#Freq_chan=dataT['DATA'].shape[4] #no of frequency channels #np.tile to repeat the same value
# for i in range(Nrea):
#     for k in range(pol):
#         hdulist[0].data["DATA"][:, 0, 0, 0,:, k, 0] =np.tile(vis[i].T.real.astype(np.float64)[:,np.newaxis],Freq_chan) 
#         hdulist[0].data["DATA"][:, 0, 0, 0,:, k, 1] =np.tile(vis[i].T.imag.astype(np.float64)[:,np.newaxis],Freq_chan)
#         if uaps:
#             op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_UAPS_{i+1}.fits"
#         else:
#             op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_APS_{Amp}_beta{beta}_{i+1}.fits"
#         hdulist.writeto(op_filename, overwrite=True)


#data output for multiple realization in same file along the frequency channels
# for j in range(pol):
#     hdulist[0].data['DATA'][:,0,0,0,:Nrea,j, 0] = vis.T.real.astype(np.float64)
#     hdulist[0].data['DATA'][:,0,0,0,:Nrea,j, 1] = vis.T.imag.astype(np.float64)

# if uaps:
#     op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_UAPS_{i+1}.fits"
# else:
#     op_filename = f"{output_dir}/Nside_{nside}_RA_{ra_ptg}_APS_{Amp}_beta{beta}_{i+1}.fits"

# hdulist.writeto(op_filename,overwrite = True) # note that the files will be overwritten


# hdulist.close()
end2 = time.time()
hours, rem = divmod(end2-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Took time all total",end='\t')
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))


APS simulation for Amp = 100 and beta = -2
10 Sky maps generated.
Took time for generating GRF	 00:00:00.01
(10, 12288)
RA = 0
DEC = -26.79522222222223
(7875, 3)
Nside= 32 resolution= 109.93556517815699 min 12288 pixels
Time taken To calculate Beam Pattern for RA 0.0 , Dec -26.79522222222223
00:00:00.00
1.943546191247974
0.7306667566629442
(10, 6144)
00:00:00.61
(10, 7875)
Took time all total	00:00:00.62
